# Word2Vec Research

## Starting project research - 24/09/2020

#### Beginning Research for Honours Project by implementing Word2Vec myself.

The motivation behind this research is based on the assumption that graphemes and phonemes adhere to the distributional hypothesis in the same way words do. That is, the meaning of graphemes (and phonemes) lie within the words that surround them (context). I am hoping that if I can build a neural network model using Tensorflow for to implement Word2Vec, I can also design a similar model to create grapheme embeddings, and then hopefully phoneme embeddings.

For this research I will use the bible.txt from project guntenberg

#### Import Modules

In [2]:
from nltk.tokenize import word_tokenize
import numpy as np
import re
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


#### Open Text File

Open text file and add each tokenized sentence to data array.

In [60]:
data = []
with open("data/bbc_samp.adj",'r') as file:
    file = re.sub(r'[^a-zA-Z\s]','',file.read())
    for line in file.split('\n'):
        tokens = word_tokenize(line.lower())
        if len(tokens) > 5:
            data.append(tokens)

#### Build a Vocabulary

Go through the data and add each unique word to a vobabulary list.

In [61]:
vocab = list(set([word for sent in data for word in sent]))
vocab.sort()
v = len(vocab)

#### Create One-Hot vectors for Training Data
For each word in the vocabulary create a one hot vector and map to corresponding word.

In [62]:
def word2int(word):
    return vocab.index(word)
    
def int2word(index):
    return vocab[index]

def int2vec(integer):
    vec=np.zeros(len(vocab))
    vec[integer]=1
    return vec

def word2vec(word):
    return (int2vec(word2int(word)))

### Create Training data

Iterate through each line and for each word in the line, place a window around the word.
Create the one hot vectors for words n to the left of the word and n to the right of the word.
place these in a row vector and add to training input set. 
Then convert the centre word to a one hot and add it to the label data.

In [93]:
window_size=2
# use window_size words before and after the context word
c=0
for line in data:
   c +=len(line)-window_size*2 

c=c*(window_size*2)

x_trn = np.empty(shape=(c,len(vocab)))
y_trn = np.empty(shape=(c,len(vocab)))

data_N = len(data)


i = 0
for line in data:
    wc = len(line)
    for term_i in range(2,wc-2):
        x_trn[i:i+window_size*2,:]=word2vec(line[term_i]) 
        y_trn[i,:] = word2vec(line[term_i-2])
        y_trn[i+1,:] = word2vec(line[term_i-1])
        y_trn[i+2,:] = word2vec(line[term_i+1])
        y_trn[i+3,:] = word2vec(line[term_i+2])
        i+=window_size*2


In [103]:
print(int2word(int(np.multiply(y_trn[0,13030:13030*2],np.arange(13030)).sum())))
print(data[1])
x_trn.shape

old
['the', 'first', 'book', 'of', 'moses', 'called', 'genesis']


(10000, 13030)

### Setting up the model architecture

In [94]:
x = tf.placeholder(tf.float32, shape=(None, v))
y = tf.placeholder(tf.float32, shape=(None, v))

embedding_dim = 15

W1 = tf.Variable(tf.random_normal([v,embedding_dim]))
b1 = tf.Variable(tf.random_normal([embedding_dim]))

hidden_layer = tf.add(tf.matmul(x,W1),b1)

W2 = tf.Variable(tf.random_normal([embedding_dim,v]))
b2 = tf.Variable(tf.random_normal([v]))

prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

### Training

In [1]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(prediction), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)
n_iters = 10000
# train for n_iter iterations
for _ in range(n_iters):
    sess.run(train_step, feed_dict={x: x_trn, y: y_trn})
    print('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: x_trn, y: y_trn}))

NameError: name 'tf' is not defined

## TODO

Maybe get a new dataset with longer sentences.
Create training data - one word as input (x: a D dimension vector) and multiple words as output (y: stacked on top CxD dimension vector where C is training window).



In [34]:
a = np.empty(shape=(4,2))
b= np.array([1,2])
a[0:4]=b
a

array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]])